In [1]:
import csv
train, dev, test = [], [], []

In [2]:
with open('pnli_train.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        train.append(x)
print (len(train))
print (train[:3])

5983
[['Sometimes do exercise.', 'A person typically desire healthy life.', '1'], ['Who eats junk foods.', 'A person typically desire healthy life.', '0'], ['A person is sick.', 'A person typically desire healthy life.', '1']]


In [3]:
with open('pnli_dev.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        dev.append(x)
print (len(dev))
print (dev[:3])

1055
[['A person is looking for accuracy.', 'A person typically desires accurate results.', '1'], ['A person does not care for accuracy.', 'A person typically desires accurate results.', '0'], ['The person double checks their data.', 'A person typically desires accurate results.', '1']]


In [4]:
with open('pnli_test_unlabeled.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[0] and x[1] will be the sentence pairs.
        test.append(x)
print (len(test))
print (test[:3])

4850
[['The people want to have a romantic and pleasant feel.', 'People typically does desire to smell violets.'], ['The contract is to buy products from you.', 'Getting contract typically cause to make money or spend money.'], ['Train station is closed.', 'Line can typically be used to move train along tracks.']]


In [5]:
import pandas as pd
train_data=pd.DataFrame(train,columns=["precondition","statement","labels"])
c=0
labels=train_data["labels"]
labels=list(labels)
for i in range(len(labels)):
  if(labels[i]=="1"):
    labels[i]="entailment"
  else:
    labels[i]="contradiction"
train_data["label_val"]=labels
train_data.head()

,precondition,statement,labels,label_val
0,Sometimes do exercise.,A person typically desire healthy life.,1,entailment
1,Who eats junk foods.,A person typically desire healthy life.,0,contradiction
2,A person is sick.,A person typically desire healthy life.,1,entailment
3,A person is dead.,A person typically desire healthy life.,0,contradiction
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,entailment


In [6]:
train_data["labels"] = train_data["labels"].astype(str).astype(int)

In [7]:
dev_data=pd.DataFrame(dev,columns=["precondition","statement","labels"])
dev_data.head()
labels=dev_data["labels"]
labels=list(labels)
for i in range(len(labels)):
  if(labels[i]=="1"):
    labels[i]="entailment"
  else:
    labels[i]="contradiction"
dev_data["label_val"]=labels
dev_data.head()

,precondition,statement,labels,label_val
0,A person is looking for accuracy.,A person typically desires accurate results.,1,entailment
1,A person does not care for accuracy.,A person typically desires accurate results.,0,contradiction
2,The person double checks their data.,A person typically desires accurate results.,1,entailment
3,The person speeds through the experiment.,A person typically desires accurate results.,0,contradiction
4,A person is studying well.,A person typically desires accurate results.,1,entailment


In [8]:
dev_data["labels"] = dev_data["labels"].astype(str).astype(int)

In [9]:
test_data=pd.DataFrame(test,columns=["precondition","statement"])
test_data.head()

,precondition,statement
0,The people want to have a romantic and pleasan...,People typically does desire to smell violets.
1,The contract is to buy products from you.,Getting contract typically cause to make money...
2,Train station is closed.,Line can typically be used to move train along...
3,There is no water for driving the boats.,People typically desires drive boats for fun.
4,The poet is busy.,Poet can typically be used for creating poetry.


In [ ]:
!pip install --no-cache-dir transformers sentencepiece

In [11]:
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
model_name ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=False)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [12]:
def encode_premise_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
    return tokenizer.convert_tokens_to_ids(tokens)
encode_premise_sentence("paras")

[3, 121, 7]

In [13]:
def encode_hypo_sentence(s):
    tokens=[]
    tokens.append('[sep]')
    tokens+=list(tokenizer.tokenize(s))
    tokens.append('[sep]')
    return tokenizer.convert_tokens_to_ids(tokens)
encode_premise_sentence("tusharmittal")

[3, 101852, 147, 4007, 1803]

In [14]:
tokenized=[]
for i in range(len(train_data)):
    pre=encode_premise_sentence(train_data['precondition'][i])
    hyp=encode_hypo_sentence(train_data['statement'][i])
    tokenized.append(pre+hyp)
train_data['tokenized']=tokenized
train_data.head()

,precondition,statement,labels,label_val,tokenized
0,Sometimes do exercise.,A person typically desire healthy life.,1,entailment,"[3, 160193, 54, 81979, 5, 3, 62, 3445, 205794,..."
1,Who eats junk foods.,A person typically desire healthy life.,0,contradiction,"[3, 40469, 28, 9821, 6, 34002, 15381, 7, 5, 3,..."
2,A person is sick.,A person typically desire healthy life.,1,entailment,"[3, 62, 3445, 83, 133054, 5, 3, 62, 3445, 2057..."
3,A person is dead.,A person typically desire healthy life.,0,contradiction,"[3, 62, 3445, 83, 103494, 5, 3, 62, 3445, 2057..."
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,entailment,"[3, 62, 3445, 28, 9821, 155965, 136, 54, 81979..."


In [15]:
tokenized=[]
for i in range(len(dev_data)):
    pre=encode_premise_sentence(dev_data['precondition'][i])
    hyp=encode_hypo_sentence(dev_data['statement'][i])
    tokenized.append(pre+hyp)
dev_data['tokenized']=tokenized
dev_data.head()

,precondition,statement,labels,label_val,tokenized
0,A person is looking for accuracy.,A person typically desires accurate results.,1,entailment,"[3, 62, 3445, 83, 16487, 100, 61689, 219, 2408..."
1,A person does not care for accuracy.,A person typically desires accurate results.,0,contradiction,"[3, 62, 3445, 14602, 959, 517, 100, 61689, 219..."
2,The person double checks their data.,A person typically desires accurate results.,1,entailment,"[3, 581, 3445, 41929, 12765, 7, 2363, 2053, 5,..."
3,The person speeds through the experiment.,A person typically desires accurate results.,0,contradiction,"[3, 581, 3445, 38352, 7, 8305, 70, 28007, 5, 3..."
4,A person is studying well.,A person typically desires accurate results.,1,entailment,"[3, 62, 3445, 83, 35187, 214, 5299, 5, 3, 62, ..."


In [16]:
mask=[]
for i in range(len(train_data)):
    padded_seq=tokenizer(train_data['precondition'][i],train_data['statement'][i], padding=True,add_special_tokens = True)
    mask.append(padded_seq)

In [17]:
df_train_panda = train_data
df_train_panda['masked'] = mask
df_train_panda.head(5)

,precondition,statement,labels,label_val,tokenized,masked
0,Sometimes do exercise.,A person typically desire healthy life.,1,entailment,"[3, 160193, 54, 81979, 5, 3, 62, 3445, 205794,...","[input_ids, attention_mask]"
1,Who eats junk foods.,A person typically desire healthy life.,0,contradiction,"[3, 40469, 28, 9821, 6, 34002, 15381, 7, 5, 3,...","[input_ids, attention_mask]"
2,A person is sick.,A person typically desire healthy life.,1,entailment,"[3, 62, 3445, 83, 133054, 5, 3, 62, 3445, 2057...","[input_ids, attention_mask]"
3,A person is dead.,A person typically desire healthy life.,0,contradiction,"[3, 62, 3445, 83, 103494, 5, 3, 62, 3445, 2057...","[input_ids, attention_mask]"
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,entailment,"[3, 62, 3445, 28, 9821, 155965, 136, 54, 81979...","[input_ids, attention_mask]"


In [ ]:
mask=[]
for i in range(len(dev_data)):
    padded_seq=tokenizer(dev_data['precondition'][i],dev_data['statement'][i], padding=True,add_special_tokens = True)
    mask.append(padded_seq)
df_dev_panda = dev_data
df_dev_panda['masked'] = mask
df_dev_panda.head(5)

In [19]:
import tensorflow as tf

In [20]:
max_len = 40

def build_model():
    encoder = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
  
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")  
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
 
    embedding = encoder([input_word_ids, input_mask])[0] 
    output = tf.keras.layers.Dense(2, activation='softmax')(embedding[:,0,:])
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output) 
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
    
    return model

In [21]:
def input_convert(data):
    inputs={
        'input_word_ids':[],
        'input_mask':[]
    }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
    return inputs

In [ ]:
len(df_train_panda_input)

2

In [22]:
df_train_panda_input=input_convert(df_train_panda['masked'].values)
for key in df_train_panda_input.keys():
    df_train_panda_input[key] = df_train_panda_input[key][:,:max_len]

In [23]:
df_dev_panda_input=input_convert(df_dev_panda['masked'].values)
for key in df_dev_panda_input.keys():
    df_dev_panda_input[key] = df_dev_panda_input[key][:,:max_len]

In [24]:
model = build_model()

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some layers from the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing TFXLMRobertaModel: ['classifier']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, us

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 40)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 40)]         0           []                               
                                                                                                  
 tfxlm_roberta_model (TFXLMRobe  TFBaseModelOutputWi  559890432  ['input_word_ids[0][0]',         
 rtaModel)                      thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 40,                                            

In [ ]:
model.fit(df_train_panda_input,train_data['labels'],verbose=2,validation_data=(df_dev_panda_input,dev_data['labels']),epochs = 12) 

Epoch 1/12
187/187 - 207s - loss: 0.3790 - accuracy: 0.8421 - val_loss: 0.2918 - val_accuracy: 0.8777 - 207s/epoch - 1s/step
Epoch 2/12
187/187 - 177s - loss: 0.2372 - accuracy: 0.9149 - val_loss: 0.2779 - val_accuracy: 0.9024 - 177s/epoch - 949ms/step
Epoch 3/12
